In [1]:
# Configuration
file_name = "schedule.txt"
sem_start = '2020-01-'
sem_end = '20200503'
day_to_date = {"MO":27, "TU":21, "WE":22, "TH":23, "FR":24}

In [2]:
# File handling
f = open(file_name,"r")
lines = f.read().splitlines()

In [3]:
# Create Schedule by Day
Monday = lines[lines.index('Monday')+1:lines.index('Tuesday')-1]
Tuesday = lines[lines.index('Tuesday')+1:lines.index('Wednesday')-1]
Wednesday = lines[lines.index('Wednesday')+1:lines.index('Thursday')-1]
Thursday = lines[lines.index('Thursday')+1:lines.index('Friday')-1]
Friday = lines[lines.index('Friday')+1:]
schedule_by_day = [Monday, Tuesday, Wednesday, Thursday, Friday]

In [4]:
# Create master event variable
master = []
i = -1
for day in schedule_by_day:
    i = i+1
    classes = day[0::5]
    lectures = day[1::5]
    locations = day[2::5]
    times = day[3::5]
    d = ["MO", "TU", "WE", "TH", "FR"]
    for e in range(0, len(classes)):
            a = times[e].split("to")
            a[0] = a[0].strip()
            a[1] = a[1].strip()

            if 'AM' in a[0]:
                start_time = a[0].split(" ")[0]
            elif 'PM' in a[0]:
                start_time = a[0].split(" ")[0]
                if(start_time[0:2]!='12'):
                    start_time = str(int(start_time[0:start_time.index(':')])+12) + start_time[start_time.index(':'):]

            if 'AM' in a[1]:
                end_time = a[1].split(" ")[0]
            elif 'PM' in a[1]:
                end_time = a[1].split(" ")[0]
                if(end_time[0:2]!='12'):
                    end_time = str(int(end_time[0:end_time.index(':')])+12) + end_time[end_time.index(':'):]
                
            event = {
                'summary': classes[e],
                'location': locations[e] + ', Madison, WI 53706',
                'description': lectures[e],
                'start': {
                    'dateTime': sem_start + str(day_to_date[d[i]]) +'T'+start_time+':00-05:00',
                    'timeZone': 'America/Chicago',
                },
                'end': {
                    'dateTime': sem_start + str(day_to_date[d[i]]) + 'T'+end_time+':00-05:00',
                    'timeZone': 'America/Chicago',
                },
                'recurrence': [
                    'RRULE:FREQ=WEEKLY;UNTIL='+sem_end+';BYDAY='+d[i]
                ],
                'reminders': {
                    'useDefault': False,
                    'overrides': [
                      {'method': 'popup', 'minutes': 15},
                    ],
                },
            }
            master.append(event)

In [5]:
# Import Google Calendar API and define event creation function
from cal_setup import get_calendar_service


def create(event):
    service = get_calendar_service()

    event_result = service.events().insert(calendarId='primary',
        body=event).execute()

    print("created event")
    print("id: ", event_result['id'])
    print("summary: ", event_result['summary'])
    print("starts at: ", event_result['start']['dateTime'])
    print("ends at: ", event_result['end']['dateTime'])

ModuleNotFoundError: No module named 'googleapiclient'

In [ ]:
# Sync schedule to Google Calendar
for e in master:
    create(e)